# Set Up

In [0]:
blob_container = "blobcontainer" # The name of your container created in https://portal.azure.com
storage_account = "w261section05group03" # The name of your Storage account created in https://portal.azure.com
secret_scope = "w261section05group04" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "houseofthedragon" # The name of the secret key created in your local computer using the Databricks CLI 
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/mids-w261"

In [0]:
spark.conf.set(
  f"fs.azure.sas.{blob_container}.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)

In [0]:
from pyspark.sql.functions import col, when, to_timestamp

##Airline Reputation Feature

In [0]:
dataset = spark.read.parquet(f"{blob_url}/dev_rank_set_1")

In [0]:
display(dataset.select('OP_UNIQUE_CARRIER').distinct())#.collect()

OP_UNIQUE_CARRIER
UA
NK
AA
EV
B6
DL
OO
F9
HA
AS


In [0]:
#Display relevant columns
#continuous flight features, continuous graph features (page rank), categorical flight features
flight_features_cont = ['DISTANCE', 'PREV_DEP_DELAY', 'CRS_DEP_TIME', 'CRS_ARR_TIME','PageRank_origin','PageRank_dest']
flight_features_cat = ['DAY_OF_MONTH']
target_col = ['label']

#dataset_sample = dataset.select('DISTANCE', 'PREV_DEP_DELAY', 'CRS_DEP_TIME', 'CRS_ARR_TIME','PageRank_origin','PageRank_dest','DAY_OF_MONTH','label').limit(10)
dataset_sample = dataset.select('DISTANCE', 'PREV_DEP_DELAY', 'CRS_DEP_TIME', 'CRS_ARR_TIME','PageRank_origin','PageRank_dest','DAY_OF_MONTH','label')
#dataset_sample = dataset.withColumns(features_to_keep,target_col).limit(10)
display(dataset_sample)
#display(dataset)

DISTANCE,PREV_DEP_DELAY,CRS_DEP_TIME,CRS_ARR_TIME,PageRank_origin,PageRank_dest,DAY_OF_MONTH,label
406.0,1.0,1905.0,2031.0,0.06604546474689019,0.01057108206796489,31,1.0
508.0,0.0,1905.0,2051.0,0.06604546474689019,0.0010472549080134436,31,0.0
551.0,0.0,1905.0,2002.0,0.06604546474689019,0.0011619875019345857,31,1.0
270.0,15.0,1905.0,2015.0,0.06604546474689019,0.003286315932802374,31,0.0
606.0,0.0,1905.0,2015.0,0.06604546474689019,0.054545842038817516,31,1.0
563.0,0.0,1905.0,2000.0,0.06604546474689019,9.92030384349354E-4,31,0.0
577.0,0.0,1905.0,2053.0,0.06604546474689019,0.016304818575501732,31,0.0
270.0,6.0,1905.0,2011.0,0.06604546474689019,0.003286315932802374,31,0.0
332.0,0.0,1905.0,1929.0,0.06604546474689019,0.0026759716363659604,31,0.0
581.0,1.0,1905.0,2056.0,0.06604546474689019,0.011872350109994463,31,0.0


In [0]:
# Additional Flight features

#M&A table: https://www.airlines.org/dataset/u-s-airline-mergers-and-acquisitions/
#
#table location, but couldn't figure out how to import it
#airline_revenue_rankings = hive_metastore.default.airline_rankings_2019
#airline_revenue_ranking.write.mode("overwrite").parquet(f"{blob_url}/airline_revenue_ranking")
df2 = spark.read.csv("hive_metastore.default.airline_rankings_2019.csv")

#manual creation of table
#columns = ['Airline Code','Operating Revenue','Operating Revenue Rank']
#data = ()

#NPS table



---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
<command-1215577238242690> in <cell line: 8>()
      6 #airline_revenue_rankings = hive_metastore.default.airline_rankings_2019
      7 #airline_revenue_ranking.write.mode("overwrite").parquet(f"{blob_url}/airline_revenue_ranking")
----> 8 df2 = spark.read.csv("hive_metastore.default.airline_rankings_2019.csv")
      9 
     10 #columns = ['Airline Code','Operating Revenue','Operating Revenue Rank']

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/sql/readwriter.py in csv(self, path, schema, sep

In [0]:
!pip install gdacs-api
from gdacs.api import GDACSAPIReader

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-27133c6d-6bf0-4d4b-a0ea-0c5f1cd0ccfa/bin/python -m pip install --upgrade pip' command.


In [0]:
#Explore natural disasters
client = GDACSAPIReader()
events = client.latest_events(limit=10)

In [0]:
display(events)

GeoJSON(type='FeatureCollection', features=[{'type': 'Feature', 'bbox': [-69.5322, -17.7013, -69.5322, -17.7013], 'geometry': {'type': 'Point', 'coordinates': [-69.5322, -17.7013]}, 'properties': {'eventtype': 'EQ', 'eventid': 1348945, 'episodeid': 1476726, 'eventname': '', 'glide': '', 'name': 'Earthquake in Chile', 'description': 'Earthquake in Chile', 'htmldescription': 'Green M 4.8 Earthquake in Chile at: 03 Dec 2022 11:45:29.', 'icon': 'https://www.gdacs.org/images/gdacs_icons/maps/Green/EQ.png', 'iconoverall': 'https://www.gdacs.org/images/gdacs_icons/maps/Green/EQ.png', 'url': {'geometry': 'https://www.gdacs.org/gdacsapi/api/polygons/getgeometry?eventtype=EQ&eventid=1348945&episodeid=1476726', 'report': 'https://www.gdacs.org/report.aspx?eventid=1348945&episodeid=1476726&eventtype=EQ', 'details': 'https://www.gdacs.org/gdacsapi/api/events/geteventdata?eventtype=EQ&eventid=1348945'}, 'alertlevel': 'Green', 'alertscore': 1, 'episodealertlevel': 'Green', 'episodealertscore': 0.0, '